# Import data

In [1]:
import pandas as pd

df = pd.read_csv('winequality-red.csv')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
# Split training set and test set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch

X = df.drop('quality', axis=1).values
y = df['quality'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Creating a basic MLP

In [6]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Making changes to basic MLP

In [7]:
import torch.optim as optim

def train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val, epochs=100):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train).squeeze()
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val).squeeze()
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss.item())

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

    return train_losses, val_losses

model = MLP()

In [8]:
# Original hyperparameters
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)
train_losses, val_losses = train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val)

model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze()
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE: {test_loss.item():.4f}')

/Users/shelger/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [10/100], Loss: 35.5854, Val Loss: 36.0617
Epoch [20/100], Loss: 35.4117, Val Loss: 35.8851
Epoch [30/100], Loss: 35.2391, Val Loss: 35.7096
Epoch [40/100], Loss: 35.0678, Val Loss: 35.5352
Epoch [50/100], Loss: 34.8975, Val Loss: 35.3620
Epoch [60/100], Loss: 34.7283, Val Loss: 35.1898
Epoch [70/100], Loss: 34.5604, Val Loss: 35.0188
Epoch [80/100], Loss: 34.3936, Val Loss: 34.8490
Epoch [90/100], Loss: 34.2278, Val Loss: 34.6801
Epoch [100/100], Loss: 34.0631, Val Loss: 34.5123
Test MSE: 34.5570


In [11]:
# increase epochs and reduce learning rate
model = MLP()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
train_losses, val_losses = train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val, 300)

model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze()
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE: {test_loss.item():.4f}')

Epoch [10/300], Loss: 31.7346, Val Loss: 31.9421
Epoch [20/300], Loss: 30.0132, Val Loss: 30.2020
Epoch [30/300], Loss: 28.3856, Val Loss: 28.5540
Epoch [40/300], Loss: 26.8375, Val Loss: 26.9842
Epoch [50/300], Loss: 25.3589, Val Loss: 25.4816
Epoch [60/300], Loss: 23.9420, Val Loss: 24.0410
Epoch [70/300], Loss: 22.5828, Val Loss: 22.6587
Epoch [80/300], Loss: 21.2804, Val Loss: 21.3347
Epoch [90/300], Loss: 20.0353, Val Loss: 20.0689
Epoch [100/300], Loss: 18.8500, Val Loss: 18.8643
Epoch [110/300], Loss: 17.7245, Val Loss: 17.7225
Epoch [120/300], Loss: 16.6627, Val Loss: 16.6462
Epoch [130/300], Loss: 15.6665, Val Loss: 15.6367
Epoch [140/300], Loss: 14.7363, Val Loss: 14.6943
Epoch [150/300], Loss: 13.8732, Val Loss: 13.8217
Epoch [160/300], Loss: 13.0758, Val Loss: 13.0180
Epoch [170/300], Loss: 12.3408, Val Loss: 12.2794
Epoch [180/300], Loss: 11.6666, Val Loss: 11.6020
Epoch [190/300], Loss: 11.0456, Val Loss: 10.9787
Epoch [200/300], Loss: 10.4764, Val Loss: 10.4072
Epoch [21

In [12]:
# keep increasing epoch and reducing learning rate
# to check if overfitting happens
model = MLP()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
train_losses, val_losses = train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val, 600)

model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze()
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE: {test_loss.item():.4f}')

Epoch [10/600], Loss: 19.9152, Val Loss: 18.8316
Epoch [20/600], Loss: 9.6243, Val Loss: 8.7739
Epoch [30/600], Loss: 4.4197, Val Loss: 3.8817
Epoch [40/600], Loss: 1.5210, Val Loss: 1.2454
Epoch [50/600], Loss: 0.6936, Val Loss: 0.6133
Epoch [60/600], Loss: 0.5394, Val Loss: 0.4947
Epoch [70/600], Loss: 0.4841, Val Loss: 0.4511
Epoch [80/600], Loss: 0.4587, Val Loss: 0.4316
Epoch [90/600], Loss: 0.4458, Val Loss: 0.4225
Epoch [100/600], Loss: 0.4387, Val Loss: 0.4180
Epoch [110/600], Loss: 0.4346, Val Loss: 0.4157
Epoch [120/600], Loss: 0.4320, Val Loss: 0.4145
Epoch [130/600], Loss: 0.4301, Val Loss: 0.4138
Epoch [140/600], Loss: 0.4287, Val Loss: 0.4134
Epoch [150/600], Loss: 0.4276, Val Loss: 0.4131
Epoch [160/600], Loss: 0.4266, Val Loss: 0.4129
Epoch [170/600], Loss: 0.4257, Val Loss: 0.4127
Epoch [180/600], Loss: 0.4249, Val Loss: 0.4124
Epoch [190/600], Loss: 0.4242, Val Loss: 0.4121
Epoch [200/600], Loss: 0.4236, Val Loss: 0.4119
Epoch [210/600], Loss: 0.4230, Val Loss: 0.4116

In [14]:
model = MLP()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
train_losses, val_losses = train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val, 1000)

model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze()
    test_loss = criterion(predictions, y_test)
    print(f'Test MSE: {test_loss.item():.4f}')

Epoch [10/1000], Loss: 21.2051, Val Loss: 20.5321
Epoch [20/1000], Loss: 12.0663, Val Loss: 11.5530
Epoch [30/1000], Loss: 6.5643, Val Loss: 6.2887
Epoch [40/1000], Loss: 3.7720, Val Loss: 3.6058
Epoch [50/1000], Loss: 1.5153, Val Loss: 1.3946
Epoch [60/1000], Loss: 0.7006, Val Loss: 0.6911
Epoch [70/1000], Loss: 0.5479, Val Loss: 0.5496
Epoch [80/1000], Loss: 0.5019, Val Loss: 0.4997
Epoch [90/1000], Loss: 0.4777, Val Loss: 0.4721
Epoch [100/1000], Loss: 0.4622, Val Loss: 0.4544
Epoch [110/1000], Loss: 0.4517, Val Loss: 0.4425
Epoch [120/1000], Loss: 0.4444, Val Loss: 0.4342
Epoch [130/1000], Loss: 0.4393, Val Loss: 0.4283
Epoch [140/1000], Loss: 0.4357, Val Loss: 0.4240
Epoch [150/1000], Loss: 0.4331, Val Loss: 0.4209
Epoch [160/1000], Loss: 0.4312, Val Loss: 0.4186
Epoch [170/1000], Loss: 0.4297, Val Loss: 0.4168
Epoch [180/1000], Loss: 0.4287, Val Loss: 0.4154
Epoch [190/1000], Loss: 0.4278, Val Loss: 0.4143
Epoch [200/1000], Loss: 0.4272, Val Loss: 0.4135
Epoch [210/1000], Loss: 0

# Optional: Implementing the MLP

In [5]:
# Add steps